In [33]:
import pandas as pd
import requests
dodman_latlong = [50.220564,-4.801677]
example_endpoint = f'https://api.open-meteo.com/v1/forecast?latitude={dodman_latlong[0]}&longitude={dodman_latlong[0]}&hourly=temperature_2m,precipitation_probability,rain,pressure_msl,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,windgusts_10m&daily=sunrise,sunset&timezone=Europe%2FLondon'
test = requests.get(example_endpoint)
print(type(test.json()))
print(test.json().keys())
hourly = pd.DataFrame(test.json()['hourly'], columns=test.json()['hourly_units'])
daily = pd.DataFrame(test.json()['daily'],columns=test.json()['daily_units'])

print(daily.head())

<class 'dict'>
dict_keys(['latitude', 'longitude', 'generationtime_ms', 'utc_offset_seconds', 'timezone', 'timezone_abbreviation', 'elevation', 'hourly_units', 'hourly', 'daily_units', 'daily'])
         time           sunrise            sunset
0  2023-09-16  2023-09-16T03:15  2023-09-16T15:52
1  2023-09-17  2023-09-17T03:17  2023-09-17T15:50
2  2023-09-18  2023-09-18T03:18  2023-09-18T15:47
3  2023-09-19  2023-09-19T03:20  2023-09-19T15:45
4  2023-09-20  2023-09-20T03:21  2023-09-20T15:43


## print out the entire dict

In [3]:
print(test.json().keys())

dict_keys(['latitude', 'longitude', 'generationtime_ms', 'utc_offset_seconds', 'timezone', 'timezone_abbreviation', 'elevation', 'hourly_units', 'hourly', 'daily_units', 'daily'])


## print hourly

In [14]:
hourly.head(5)

,time,temperature_2m,precipitation_probability,rain,pressure_msl,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,windgusts_10m
0,2023-09-16T00:00,10.5,17,0.0,1019.6,36,49,100,4.7,9,7.9
1,2023-09-16T01:00,10.3,19,0.0,1019.7,5,25,10,3.8,49,8.3
2,2023-09-16T02:00,10.0,16,0.0,1019.8,9,10,6,3.4,32,6.8
3,2023-09-16T03:00,9.5,13,0.0,1020.1,41,23,0,3.3,6,5.0
4,2023-09-16T04:00,9.3,10,0.0,1020.0,61,3,0,4.0,355,8.3


## Get timestamp, and return appropriate column

In [36]:
import datetime
import pandas as pd
import requests

def get_hourly_weather_from_timestamp(endpoint):
    """Returns the correct row from the hourly dataframe for a given timestamp"""
    test = requests.get(example_endpoint)
    current_time = datetime.datetime.now()
    formatted_time = current_time.strftime('%Y-%m-%dT%H:00')
    hourly = pd.DataFrame(test.json()['hourly'], columns=test.json()['hourly_units'])
    matching_row = hourly[hourly['time'] == formatted_time].reset_index(drop=True)
    matching_row = matching_row.values[0]
    return_time = current_time.strftime('%Y-%m-%dT%H:%M:%S')
    matching_row[0] = return_time
   
    return matching_row

def get_daily_weather_from_timestamp(endpoint):
    """Returns the correct row from the daily dataframe for a given timestamp"""
    test = requests.get(example_endpoint)
    current_time = datetime.datetime.now()
    formatted_date_str = current_time.strftime('%Y-%m-%d')
    matching_row_daily = daily[daily['time'] == formatted_date_str].reset_index(drop=True)
    matching_row_daily = matching_row_daily.values[0]
    return_date = current_time.strftime('%Y-%m-%dT%H:%M:%S')
    matching_row_daily[0] = return_date
    return matching_row_daily

def get_weather_col_names(endpoint, hourly_flag=True):
    """Returns the column names from the endpoint, hourly cols if hourlyflag is True, else daily cols"""
    try:
        test = requests.get(endpoint)
        test.raise_for_status()
        if hourly_flag:
            return test.json()['hourly_units']
        else:
            return test.json()['daily_units']
    except requests.exceptions.HTTPError as err:
        raise SystemExit(err)


dodman_latlong = [50.220564,-4.801677]
example_endpoint = f'https://api.open-meteo.com/v1/forecast?latitude={dodman_latlong[0]}&longitude={dodman_latlong[0]}&hourly=temperature_2m,precipitation_probability,rain,pressure_msl,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,windgusts_10m&daily=sunrise,sunset&timezone=Europe%2FLondon'
matching_row_hourly = get_hourly_weather_from_timestamp(example_endpoint)
matching_row_daily = get_daily_weather_from_timestamp(example_endpoint)

print(matching_row_hourly)
print(matching_row_daily)


['2023-09-16T10:59:28' 17.4 3 0.0 1020.2 56 88 0 10.4 2 19.8]
['2023-09-16T10:59:28' '2023-09-16T03:15' '2023-09-16T15:52']
